<h1>Customer Analytics and Customer Insight</h1>
<h2>Conjoint Analysis on Preferences and Segments in the Portable Bluetooth Speaker Market</h2>

Choice data set

In [1]:
data.cbceffect<-read.csv('choiceData_590506.csv')

data.cbceffect$price<-data.cbceffect$price/10
head(data.cbceffect)

X,id,cs,alt,choice,none,price,battery1,battery2,battery3,battery4,weight1,weight2,weight3,sound1,sound2,sound3
97,11,1,1,0,0,7,0,1,0,0,1,0,0,0,0,1
98,11,1,2,0,0,15,1,0,0,0,0,0,1,1,0,0
99,11,1,3,1,0,11,0,0,1,0,1,0,0,-1,-1,-1
100,11,1,4,0,1,0,0,0,0,0,0,0,0,0,0,0
101,11,2,1,0,0,7,0,1,0,0,0,1,0,1,0,0
102,11,2,2,0,0,9,-1,-1,-1,-1,-1,-1,-1,0,1,0


In [ ]:
library(gmnl)
data_ml_bluetooth_effect=mlogit.data(data.cbceffect, choice = "choice", shape = "long",
                                 id.var = "id", alt.var = "alt")

mxl_bluetooth <- gmnl(choice ~ 0 + none+price+battery1+battery2+battery3+battery4+
                    weight1+weight2+weight3+sound1+sound2+sound3, 
                  data = data_ml_bluetooth_effect, model = "mixl", correlation = FALSE, 
                  haltons = NULL, R = 2000, panel = TRUE, tol = 1e-12, print.level = 1,
                  ranp = c(none = "n", price = "n", battery1 = "n", battery2 = "n",battery3 = "n",battery4 = "n", 
                           weight1 = "n", weight2 = "n",weight3 = "n", sound1 = "n", sound2 = "n", sound3 = "n"))


In [ ]:
mxl_betai = effect.gmnl(mxl_bluetooth)$mean
mxl_betai<-subset(mxl_betai,mxl_betai[,2]<0)
nrow(mxl_betai)
write.csv(mxl_betai,file="ind_pref_price_neg.csv",row.names=FALSE)

indivData = read.csv('indivData_590506.csv')
indivData = indivData[which(mxl_betai$price < 0),]
write.csv(indivData_sub, "indivData_sub_price_neg.csv", row.names = F)
head(mxl_betai)

In [10]:
library(ggplot2)
library(reshape2)
library(stringr)
library(rpart)
library(rpart.plot)
library(randomForest)

ind_pref = read.csv("ind_pref_price_neg.csv")
cols = names(ind_pref)

add_ommited_pw = function(ind_pref, att_name, cols){
    #adds part worths(pw) for the ommited attribute levels to the preference dataframe
  attribute_cols = cols[grep(att_name, cols)]
  new_col_name =  paste0(att_name, as.character(max(grep("[0-9]", attribute_cols)) + 1))
  ind_pref[, new_col_name] =  0 - rowSums(ind_pref[,attribute_cols])
  return(ind_pref)
}  

for(att in c("battery", "weight", "sound")){
  ind_pref = add_ommited_pw(ind_pref, att, cols = cols)
}

PW = ind_pref[,3:15]
PW = PW[, order(names(PW))]
PW = cbind(ind_pref[,1:2], PW)
head(PW)

none,price,battery1,battery2,battery3,battery4,weight1,weight2,weight3,sound1,sound2,sound3,battery5,weight4,sound4
-12.4370428,-0.56168985,-1.6588656,-0.2015096,0.1884520,0.5537557,0.7376991,0.2381162,-0.09342943,-3.0833268,-0.4324235,0.9010875,1.1181675,-0.8823858,2.6146628
-12.4982362,-0.64491141,-0.7408493,-0.2046929,0.2118804,0.5534250,0.4516939,0.1587443,0.04016417,-2.8379613,-0.3882430,0.9713326,0.1802369,-0.6506024,2.2548717
-10.5555664,-0.05169533,-1.3462516,-0.1926465,0.2096073,0.5834711,0.7291077,0.3308254,-0.02169741,-3.4474058,-0.8067364,0.8953400,0.7458196,-1.0382357,3.3588023
-5.1263237,-0.36217551,-1.3481889,-0.2574588,0.2076824,0.5167001,0.6833427,0.4968326,0.02483880,-2.9199103,-0.8453724,1.0041165,0.8812652,-1.2050141,2.7611662
-13.2687279,-0.74194118,-1.6332140,-0.1998363,0.1576263,0.5582876,1.0632011,0.2196911,-0.11231526,-1.4741027,-0.2798607,0.9185096,1.1171364,-1.1705769,0.8354537
0.4547102,-0.21835404,-1.6371675,-0.2615920,0.2387926,0.6802894,0.8795875,0.3352366,-0.23511110,-0.8796407,-0.2930464,0.8626974,0.9796775,-0.9797130,0.3099897


none,price,battery1,battery2,battery3,battery4,battery5,sound1,sound2,sound3,sound4,weight1,weight2,weight3,weight4
-12.4370428,-0.56168985,-1.6588656,-0.2015096,0.1884520,0.5537557,1.1181675,-3.0833268,-0.4324235,0.9010875,2.6146628,0.7376991,0.2381162,-0.09342943,-0.8823858
-12.4982362,-0.64491141,-0.7408493,-0.2046929,0.2118804,0.5534250,0.1802369,-2.8379613,-0.3882430,0.9713326,2.2548717,0.4516939,0.1587443,0.04016417,-0.6506024
-10.5555664,-0.05169533,-1.3462516,-0.1926465,0.2096073,0.5834711,0.7458196,-3.4474058,-0.8067364,0.8953400,3.3588023,0.7291077,0.3308254,-0.02169741,-1.0382357
-5.1263237,-0.36217551,-1.3481889,-0.2574588,0.2076824,0.5167001,0.8812652,-2.9199103,-0.8453724,1.0041165,2.7611662,0.6833427,0.4968326,0.02483880,-1.2050141
-13.2687279,-0.74194118,-1.6332140,-0.1998363,0.1576263,0.5582876,1.1171364,-1.4741027,-0.2798607,0.9185096,0.8354537,1.0632011,0.2196911,-0.11231526,-1.1705769
0.4547102,-0.21835404,-1.6371675,-0.2615920,0.2387926,0.6802894,0.9796775,-0.8796407,-0.2930464,0.8626974,0.3099897,0.8795875,0.3352366,-0.23511110,-0.9797130


In [8]:
#scale price coef to represent utilitz change per 10 USD price change
price.imp = 80/10 * abs(ind_pref$price)
importance = data.frame(price.imp)

cols = names(ind_pref)
for(att in c("battery", "weight", "sound")){
  attribute_cols = cols[grep(att, cols)]
  range_name = paste0(att, '.imp')
  importance[,range_name] = apply(ind_pref[,attribute_cols],1,function(x) abs(max(x) - min(x)))
}

rs = rowSums(importance)
rel.imp = apply(importance,2, function(x) x/rs)
rel.imp = data.frame(rel.imp)

ind_pref = cbind(ind_pref, rel.imp)
head(rel.imp)

price.imp,battery.imp,weight.imp,sound.imp
0.30801521,0.1903560,0.11105123,0.3905775
0.40789120,0.1023247,0.08714704,0.4026370
0.03732789,0.1888290,0.15951925,0.6143239
0.22820869,0.1755988,0.14873294,0.4474595
0.44586906,0.2066027,0.16779843,0.1797298
0.21930483,0.3285300,0.23342458,0.2187406


In [9]:
#Calculate Willingness to pay 

for(att in c("battery", "weight", "sound")){
  attribute_cols = cols[grep(att, cols)]
  
  for(col in attribute_cols){
    #get WTP per column by deviding through ommited column and multiplying with price coef
    col_WTP = (ind_pref[,col] - ind_pref[,attribute_cols[length(attribute_cols)]]) * abs(ind_pref$price) * 10
    
    wtp_name = paste0(col, ".WTP")
    ind_pref[,wtp_name] = col_WTP
  }
  
  
  range_name = paste0(att, '.imp')
  importance[,range_name] = apply(ind_pref[,attribute_cols],1,function(x) abs(max(x) - min(x)))
}

WTP = ind_pref[,names(ind_pref)[grep("WTP", names(ind_pref))]]
#WTP = cbind(clust = ind_pref$clust, WTP)
head(WTP)

battery1.WTP,battery2.WTP,battery3.WTP,battery4.WTP,battery5.WTP,battery.imp.WTP,weight1.WTP,weight2.WTP,weight3.WTP,weight4.WTP,weight.imp.WTP,sound1.WTP,sound2.WTP,sound3.WTP,sound4.WTP,sound.imp.WTP
-10.386890,-2.201070,-0.01069482,2.0411790,5.2114228,0,3.5198174,0.7137110,-1.14854713,-5.5800352,0,-19.512568,-4.6227131,2.8674826,12.492461,0
-5.437726,-1.979992,0.70653677,2.9091969,0.5024642,0,2.3510045,0.4617389,-0.30299786,-4.7578303,0,-20.898988,-5.1004755,3.6675825,11.945273,0
-0.793565,-0.197205,0.01074143,0.2040115,0.2879382,0,0.2944506,0.0885573,-0.09368056,-0.6191834,0,-2.099725,-0.7346219,0.1452722,1.418767,0
-5.518786,-1.568429,0.11619892,1.2353853,2.5557506,0,1.9362256,1.2607316,-0.44871421,-4.9029400,0,-12.195789,-4.6823205,2.0160753,8.379679,0
-13.650357,-3.015538,-0.36337601,2.6092954,6.7556250,0,6.6433613,0.3850129,-2.07827882,-9.9299579,0,-12.270464,-3.4098918,5.4813116,4.865086,0
-4.292180,-1.288555,-0.19594522,0.7680809,1.4218070,0,1.4109229,0.2223106,-1.02306657,-2.6489349,0,-2.398360,-1.1175076,1.4061057,0.199246,0
